In [1]:
from typing import List, Optional
import dotenv
from dataclasses import dataclass, field
import os
import time

import discogs_client
from discogs_client.exceptions import HTTPError

In [2]:
@dataclass()
class Node:
    name: str
    discogs_id: int
    page_url: str = None


@dataclass()
class ReleaseNode(Node):
    year: int = None


@dataclass()
class ArtistNode(Node):
    image_url: str = None
    releases: List[ReleaseNode] = field(default_factory=list)

In [3]:
@dataclass()
class DiscoConnector:
    client: discogs_client.Client = None
    auth_url: str = None
    request_token: str = None
    request_secret: str = None
    token: str = None
    secret: str = None

    def __init__(self, key, secret):
        self.client = discogs_client.Client(
            "MaliRobot/1.0",
            consumer_key=key,
            consumer_secret=secret
        )

    def get_new_token(self):
        self.request_token, self.request_secret, self.auth_url = self.client.get_authorize_url()

        accepted = 'n'
        while accepted.lower() == 'n':
            print("\n")
            accepted = input(f'Have you authorized me at {self.auth_url} [y/n] :')

        oauth_verifier = input('Verification code : ')
        token, secret = self.client.get_access_token(oauth_verifier)
        self.set_token(token, secret)

    def search(self, term: str, type: Optional[str]):
        if self.token is None:
            self.get_new_token()
        try:
            return self.client.search(term, type=type)
        except discogs_client.exceptions.HTTPError:
            self.get_new_token()
            self.search(term, type)

    def set_token(self, token, secret):
        self.token = token
        self.secret = secret
        self.client.set_token(self.token, self.secret)

    def get_release(self, release_id):
        return self.client.release(release_id)

    def get_artist(self, artist_id):
        return self.client.artist(artist_id)

    def search_artist(self, artist: str):
        results = self.search(artist, type="artist")
        for r in results:
            if r.name == artist:
                artist_node = ArtistNode(
                    name=r.name,
                    discogs_id=r.id,
                    page_url=r.url,
                )

                if len(r.images) > 0:
                    artist_node.image_url = r.images[0]['uri']

                return artist_node, r
        return None, None

In [4]:
    dotenv.load_dotenv()
    term = "Adam Green"

    client = DiscoConnector(
        key=os.getenv("DISCOGS_KEY"),
        secret=os.getenv("DISCOGS_SECRET")
    )
    client.set_token("AQarrhoMZGLUqElIiJAEXwlIbIwEBBOYwhqfkktT", "hAmOQhiaFTjPGnxMuiMGlyvOiCyYGYTqbTLRSqJB")

    # client = DiscoSimpleConnector(
    #     token="TiaUbddZcZbgJVZPMYmHALvIivJKOZUEWVQCMRaz"
    # )
    start = time.time()
    artist_node, artist = client.search_artist(term)
    if not artist:
        print("no artist")


In [5]:
print(artist)

<Artist 138069 'Adam Green'>


In [6]:
rels = artist.releases
print(dir(rels))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_filters', '_invalidate', '_list_key', '_load_pagination_info', '_num_items', '_num_pages', '_pages', '_per_page', '_sort_key', '_sort_order', '_transform', '_url_for_page', 'client', 'count', 'filter', 'page', 'pages', 'per_page', 'sort', 'url']


In [7]:
print([x for x in rels.page(0)])

[<Master 2412817 'Dance With Me'>, <Master 368237 'Garfield'>, <Master 141963 'Jessica / Kokomo'>, <Master 109277 'Friends Of Mine'>, <Master 141962 'Emily'>, <Master 237363 'Carolina*'>, <Master 109282 'Friends Of Mine'>, <Release 3228008 'Gemstones EPK'>, <Release 5680502 'Gemstones (Album Sampler)'>, <Master 109279 'Gemstones*'>, <Release 19914556 'Tour Sampler'>, <Master 212516 'Nat King Cole'>, <Master 109281 'Jacket Full Of Danger'>, <Release 21278101 'Morning After Midnight (Pixelated Version)'>, <Master 1142807 'Daytrotter Studio 8/5/2008'>, <Master 109280 'Sixes & Sevens'>, <Master 1752951 'Morning After Midnight'>, <Master 1752956 'Twee Twee Dee'>, <Master 335803 'Minor Love'>, <Master 372178 'I Will'>, <Master 1295849 'What Makes Him Act So Bad'>, <Release 2548208 'Musik For A Play'>, <Master 1752954 'Buddy Bradley'>, <Master 517717 'Adam Green & Binki Shapiro'>, <Release 5040522 'Pity Love'>, <Release 9908935 'Studio Paradiso 2/20/2013'>, <Master 1752961 'Nature Of The Clow

In [8]:
print(dir(rels.page(0)[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_known_invalid_keys', 'changes', 'client', 'data', 'data_quality', 'delete', 'fetch', 'genres', 'id', 'images', 'main_release', 'previous_request', 'refresh', 'save', 'styles', 'title', 'tracklist', 'url', 'versions', 'videos', 'year']


In [9]:
print(rels.page(0)[0].data)

{'id': 2412817, 'title': 'Dance With Me', 'type': 'master', 'main_release': 406090, 'artist': 'Adam Green', 'role': 'Main', 'resource_url': 'https://api.discogs.com/masters/2412817', 'year': 2002, 'thumb': 'https://i.discogs.com/JfIOQ8l92EIV_Apk_dIY66HT9Q3-29fdrWh3RzvsWY0/rs:fit/g:sm/q:40/h:150/w:150/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTQwNjA5/MC0xMTA5MTYzMTg3/LmpwZw.jpeg', 'stats': {'community': {'in_wantlist': 19, 'in_collection': 60}, 'user': {'in_wantlist': 0, 'in_collection': 0}}}


In [10]:
print(rels.pages)

4


In [11]:
for r in artist.releases:
    print(r)

<Master 2412817 'Dance With Me'>
<Master 368237 'Garfield'>
<Master 141963 'Jessica / Kokomo'>
<Master 109277 'Friends Of Mine'>
<Master 141962 'Emily'>
<Master 237363 'Carolina*'>
<Master 109282 'Friends Of Mine'>
<Release 3228008 'Gemstones EPK'>
<Release 5680502 'Gemstones (Album Sampler)'>
<Master 109279 'Gemstones*'>
<Release 19914556 'Tour Sampler'>
<Master 212516 'Nat King Cole'>
<Master 109281 'Jacket Full Of Danger'>
<Release 21278101 'Morning After Midnight (Pixelated Version)'>
<Master 1142807 'Daytrotter Studio 8/5/2008'>
<Master 109280 'Sixes & Sevens'>
<Master 1752951 'Morning After Midnight'>
<Master 1752956 'Twee Twee Dee'>
<Master 335803 'Minor Love'>
<Master 372178 'I Will'>
<Master 1295849 'What Makes Him Act So Bad'>
<Release 2548208 'Musik For A Play'>
<Master 1752954 'Buddy Bradley'>
<Master 517717 'Adam Green & Binki Shapiro'>
<Release 5040522 'Pity Love'>
<Release 9908935 'Studio Paradiso 2/20/2013'>
<Master 1752961 'Nature Of The Clown'>
<Master 997196 'Aladdin

In [13]:
print(len(artist.releases))

170
170


In [23]:
release = client.get_release(1810762)
print(dir(release))
print([x.name for x in release.artists])
print(release.title)
print(release.id)
print(release.credits)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_known_invalid_keys', 'artists', 'artists_sort', 'changes', 'client', 'community', 'companies', 'country', 'credits', 'data', 'data_quality', 'delete', 'fetch', 'formats', 'genres', 'id', 'images', 'labels', 'marketplace_stats', 'master', 'notes', 'previous_request', 'price_suggestions', 'refresh', 'save', 'status', 'styles', 'thumb', 'title', 'tracklist', 'url', 'videos', 'year']
['Jay-J & Chris Lum', 'Soulfranciscins']
Give It Up (For Love)
1810762
[<Artist 14862 'Jay-J & Chris Lum'>]


In [21]:
artist = client.get_artist(709480)
print(artist.name)

Partibrejkers
